In [ ]:
# Importing required libraries
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [ ]:
# Loading pre-trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Switching environemnt to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Importing the dataset
url = 'https://raw.githubusercontent.com/yamanalab/ec-darkpattern/master/dataset/dataset.tsv'
df = pd.read_csv(url, sep="\t")
df

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency
1,158,Pillowcases & Shams,0,Not Dark Pattern
2,108,Write a review,0,Not Dark Pattern
3,1425,"To start your return, simply click on the foll...",0,Not Dark Pattern
4,1658,newsletter signup (privacy policy),0,Not Dark Pattern
...,...,...,...,...
2351,1304,Bra Panty Set,0,Not Dark Pattern
2352,1038,DressesFootwearHatsHoodies & SweatshirtsJersey...,0,Not Dark Pattern
2353,1081,Only 4 left in stock,1,Scarcity
2354,767,Trending Players,0,Not Dark Pattern


In [ ]:
# Removing irrelevant columns
df = df[['text', 'label']]
df

,text,label
0,FLASH SALE | LIMITED TIME ONLY Shop Now,1
1,Pillowcases & Shams,0
2,Write a review,0
3,"To start your return, simply click on the foll...",0
4,newsletter signup (privacy policy),0
...,...,...
2351,Bra Panty Set,0
2352,DressesFootwearHatsHoodies & SweatshirtsJersey...,0
2353,Only 4 left in stock,1
2354,Trending Players,0


In [ ]:
# Encoding the dataset
encoded = tokenizer(df.text.tolist(), truncation=True, padding=True)

In [ ]:
# Creating torch dataset
input_ids = torch.tensor(encoded['input_ids'])
attention_masks = torch.tensor(encoded['attention_mask'])
labels = torch.tensor(df.label.tolist())
dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)

In [ ]:
# Training the model
train_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, pin_memory=True)
opt = torch.optim.Adam(model.parameters(), lr=0.1)
for epoch in range(1):
   for batch in train_loader:
      batch = [t.to(device) for t in batch]
      loss = model(batch[0], attention_mask=batch[1], labels=batch[2]).loss
      loss.backward()
      opt.step()
      opt.zero_grad()

In [ ]:
# Saving the model
torch.save(model.state_dict(), "bert_model.pt")

In [ ]:
# Loading the model
loaded_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
loaded_model.load_state_dict(torch.load("bert_model.pt"))
loaded_model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Importing the test dataset
url = 'https://raw.githubusercontent.com/aruneshmathur/dark-patterns/master/data/final-dark-patterns/dark-patterns.csv'
test_df = pd.read_csv(url)
test_df

,Pattern String,Comment,Pattern Category,Pattern Type,Where in website?,Deceptive?,Website Page
0,Collin P. from Grandview Missouri just bought ...,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://alaindupetit.com/collections/all-suits...
1,"Faith in Glendale, United States purchased a C...",Periodic popup,Social Proof,Activity Notification,Product Page,No,https://bonescoffee.com/products/strawberry-ch...
2,Sharmeen Atif From Karachi just bought Stylish...,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://brandsego.com/collections/under-rs-99/...
3,9 people are viewing this.,Product detail,Social Proof,Activity Notification,Product Page,No,https://brightechshop.com/products/ambience-so...
4,5338 people viewed this in the last hour,Periodic popup,Social Proof,Activity Notification,Product Page,No,https://bumpboxes.com/
...,...,...,...,...,...,...,...
1813,$132.90 $99.00,Website adds free items to show discount,Misdirection,Visual Interference,Cart Page,No,https://www.planetofthevapes.com/products/plan...
1814,This offer is only VALID if you add to cart now!,Popup asking you to buy more,Misdirection,Visual Interference,Product Page,No,https://www.rockymountainoils.com/single-essen...
1815,NaN,Deterministic draw. Always give you the prize ...,Misdirection,Visual Interference,Product Page,Yes,https://www.sammydress.com/
1816,NaN,Shows you prices in the popup based on your cu...,Misdirection,Visual Interference,Product Page,No,https://www.shoedazzle.com/products/FEELIN-A-L...


In [ ]:
# Cleaning test dataset for null values
test_df = test_df[pd.notnull(test_df["Pattern String"])]
col = ["Pattern String", "Deceptive?"]
test_df = test_df[col]
test_df

,Pattern String,Deceptive?
0,Collin P. from Grandview Missouri just bought ...,No
1,"Faith in Glendale, United States purchased a C...",No
2,Sharmeen Atif From Karachi just bought Stylish...,No
3,9 people are viewing this.,No
4,5338 people viewed this in the last hour,No
...,...,...
1810,TWO FREE PILLOWS AND 30% OFF WITH PROMO CODE,No
1812,Compare at,No
1813,$132.90 $99.00,No
1814,This offer is only VALID if you add to cart now!,No


In [ ]:
# Formatting values in the test dataframe (Run only once)
formatted_test_df = test_df
formatted_test_df['Deceptive?'] = test_df['Deceptive?'].map({'Yes': 1, 'No': 0})
formatted_test_df

,Pattern String,Deceptive?
0,Collin P. from Grandview Missouri just bought ...,0.0
1,"Faith in Glendale, United States purchased a C...",0.0
2,Sharmeen Atif From Karachi just bought Stylish...,0.0
3,9 people are viewing this.,0.0
4,5338 people viewed this in the last hour,0.0
...,...,...
1810,TWO FREE PILLOWS AND 30% OFF WITH PROMO CODE,0.0
1812,Compare at,0.0
1813,$132.90 $99.00,0.0
1814,This offer is only VALID if you add to cart now!,0.0


In [ ]:
# Evaluating performance metrics
from sklearn.metrics import accuracy_score, classification_report
import numpy

test_texts = formatted_test_df["Pattern String"].tolist()
test_labels = numpy.array(formatted_test_df["Deceptive?"].tolist())
test_labels = [0 if numpy.isnan(x) else x for x in test_labels]
test_labels = torch.tensor(test_labels)
test_tokenized_input = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

test_input_ids = torch.tensor(test_tokenized_input['input_ids'])
test_attention_masks = torch.tensor(test_tokenized_input['attention_mask'])
test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_masks, test_labels)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False, pin_memory=True)

all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        outputs = loaded_model(batch[0], attention_mask=batch[1])
        predictions = torch.argmax(outputs.logits, dim=1)
        all_predictions.append(predictions)

all_predictions = torch.cat(all_predictions)
accuracy = accuracy_score(test_labels, all_predictions.cpu().numpy())

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(labels, all_predictions.cpu().numpy()))

<ipython-input-19-127b4e5fd709>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_input_ids = torch.tensor(test_tokenized_input['input_ids'])
<ipython-input-19-127b4e5fd709>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_attention_masks = torch.tensor(test_tokenized_input['attention_mask'])


Accuracy: 0.09259259259259259


ValueError: Found input variables with inconsistent numbers of samples: [2356, 1512]